# Simple Analysis of the 2025 NBA Trade Deadline

In [1]:
import networkx as nx
import plotly.graph_objects as go
import pandas as pd
import numpy as np

Let's get a better understanding of who got traded during the deadline, a read of all the packages that got moved, and a working understanding of how the dynamics of this NBA season may have shifted. Here are the following steps that I plan to take.
1. Pull all the trade data (Anything from start of season to Feb 6 2025)
2. Clean it up in a way that's easy to read. Then visualize the nodes teams) and the assets moved (edges).
3. See what each team's P+R+A delta may have changed.

In [2]:
trades = pd.read_csv("trades.csv")
trades

,Date,Team1,Team2,Team3,Team4,Team5
Dec 15,2024,Warriors: Dennis Schröder,Nets: De’Anthony Melton; Reece Beekman,NaN,NaN,NaN
Dec 15,2024,Pacers: Thomas Bryant,Heat: Draft Compensation,NaN,NaN,NaN
Dec 29,2024,Lakers: Dorian Finney-Smith; Shake Milton,Nets: D’Angelo Russell; Maxwell Lewis,NaN,NaN,NaN
Jan 15,2025,Suns: Nick Richards,Hornets: Josh Okogie,NaN,NaN,NaN
Feb 1,2025,Clippers: Drew Eubanks; Patty Mills,Jazz: Mo Bamba; P.J. Tucker,NaN,NaN,NaN
Feb 2,2025,Lakers: Luka Dončić; Maxi Kleber; Markieff Morris,Mavericks: Anthony Davis; Max Christie,Jazz: Jalen Hood-Schifino,NaN,NaN
Feb 2,2025,Spurs: De’Aaron Fox; Jordan McLaughlin,Kings: Zach LaVine; Sidy Cissoko,Bulls: Zach Collins; Kevin Huerter; Tre Jones,NaN,NaN
Feb 4,2025,Mavericks: Caleb Martin,76ers: Quentin Grimes,NaN,NaN,NaN
Feb 5,2025,Kings: Jonas Valančiūnas,Wizards: Sidy Cissoko,NaN,NaN,NaN
Feb 5,2025,Thunder: Daniel Theis,Pelicans: Draft Compensation,NaN,NaN,NaN


I'm going to create a directed graph where each node is a team and each edge represents a trade between two teams. I'm going to use the 3D layout from Plotly to visualize the graph. The idea here is to get a sense of all the movement across the league. This will not include trade compensation that does not involve a player.

In [5]:
# Create a directed graph
G = nx.DiGraph()

# Process each row in the trades dataframe
for _, row in trades.iterrows():
    # Get all valid team entries for this trade
    trade_teams = []
    trade_details = {}
    
    # First collect all valid team:players pairs
    for i in range(1, 6):
        team_col = f'Team{i}'
        if pd.notna(row[team_col]) and ':' in row[team_col]:
            parts = row[team_col].split(':')
            if len(parts) >= 2:
                team = parts[0].strip()
                players = parts[1].strip()
                trade_teams.append(team)
                trade_details[team] = players
    
    # Now create edges between all teams involved in this trade
    for i, team1 in enumerate(trade_teams):
        for team2 in trade_teams[i+1:]:
            # Add edge in both directions to represent the trade
            G.add_edge(team1, team2, 
                      team1_players=trade_details[team1],
                      team2_players=trade_details[team2],
                      date=row['Date'])
            G.add_edge(team2, team1,
                      team1_players=trade_details[team2],
                      team2_players=trade_details[team1],
                      date=row['Date'])

# Create 3D positions for nodes
pos = nx.spring_layout(G, dim=3, k=1/np.sqrt(len(G.nodes())), iterations=50)

# Extract node positions
node_x = [pos[node][0] for node in G.nodes()]
node_y = [pos[node][1] for node in G.nodes()]
node_z = [pos[node][2] for node in G.nodes()]

# Create edges
edge_x = []
edge_y = []
edge_z = []
edge_text = []

for edge in G.edges(data=True):
    x0, y0, z0 = pos[edge[0]]
    x1, y1, z1 = pos[edge[1]]
    
    # Add line coordinates
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])
    edge_z.extend([z0, z1, None])
    
    # Enhanced hover text with players from both teams
    hover_text = (
        f"<b>{edge[0]} ↔ {edge[1]}</b><br><br>"
        f"<b>{edge[0]} sent:</b><br>"
        f"• {edge[2]['team1_players'].replace(';', '<br>• ')}<br><br>"
        f"<b>{edge[1]} sent:</b><br>"
        f"• {edge[2]['team2_players'].replace(';', '<br>• ')}<br><br>"
        f"<i>Date: {edge[2]['date']}</i>"
    )
    edge_text.extend([hover_text, "", ""])

Here's a scatter 3D plot where the teams are represented as nodes and the moved players are represented as lines between the nodes. It's meant to be the net movement, not representation of the actual trade on an individual basis.

In [4]:
# Create edge trace
edge_trace = go.Scatter3d(
    x=edge_x, y=edge_y, z=edge_z,
    line=dict(width=1.5, color='#888'),
    hoverinfo='text',
    text=edge_text,
    mode='lines',
    hoverlabel=dict(
        bgcolor='white',
        font_size=14,
        font_family="Arial"
    )
)

# Create node trace
node_trace = go.Scatter3d(
    x=node_x, y=node_y, z=node_z,
    mode='markers+text',
    hoverinfo='text',
    text=list(G.nodes()),
    textposition="top center",
    marker=dict(
        size=10,
        color='#1f77b4',
        line=dict(width=2, color='#ffffff')
    ),
    hoverlabel=dict(
        bgcolor='lightblue',
        font_size=14,
        font_family="Arial"
    )
)

# Create the figure
fig = go.Figure(data=[edge_trace, node_trace])

In [14]:
# Update layout
fig.update_layout(
    title='NBA Trade Deadline 2025 - 3D Interactive Network',
    showlegend=False,
    hovermode='closest',
    margin=dict(b=0,l=0,r=0,t=40),
    scene=dict(
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        zaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    ),
    scene_camera=dict(
        up=dict(x=0, y=0, z=1),
        center=dict(x=0, y=0, z=0),
        eye=dict(x=1.5, y=1.5, z=1.5)
    )
)

# Show the plot
fig.show()

Let's load up the season data for each player in the nba, and match it up with the team they were on at the time of the trade. Then move it around with the team that they got moved for. The goal is to see the net movement of P+r+a.

In [20]:
player_stats = pd.read_csv("playerStats.csv")
player_stats
player_stats_dict = player_stats.set_index('NAME')['P+R+A'].to_dict()

In [22]:
import networkx as nx
import plotly.graph_objects as go
import pandas as pd
import numpy as np

trades = pd.read_csv("trades.csv")
player_stats = pd.read_csv("playerStats.csv")
player_stats_dict = player_stats.set_index('NAME')['P+R+A'].to_dict()

G = nx.Graph()
team_pra_changes = {}

for _, row in trades.iterrows():
    trade_teams = []
    trade_details = {}
    trade_pra = {}
    
    for i in range(1, 6):
        team_col = f'Team{i}'
        if pd.notna(row[team_col]) and ':' in row[team_col]:
            team, players = row[team_col].split(':', 1)
            team = team.strip()
            players_list = [p.strip() for p in players.strip().split(';')]
            
            total_pra = sum(player_stats_dict.get(player, 0) for player in players_list)
            
            trade_teams.append(team)
            trade_details[team] = players_list
            trade_pra[team] = total_pra
            
            if team not in team_pra_changes:
                team_pra_changes[team] = 0
    
    for i, team1 in enumerate(trade_teams):
        for team2 in trade_teams[i+1:]:
            G.add_edge(team1, team2, 
                       team1_players=trade_details[team1],
                       team2_players=trade_details[team2],
                       team1_pra=trade_pra[team1],
                       team2_pra=trade_pra[team2],
                       date=row['Date'])
            
            team_pra_changes[team1] += trade_pra[team2] - trade_pra[team1]
            team_pra_changes[team2] += trade_pra[team1] - trade_pra[team2]

pos = nx.spring_layout(G, dim=3, k=0.5, iterations=50)

edge_x, edge_y, edge_z = [], [], []
for edge in G.edges():
    x0, y0, z0 = pos[edge[0]]
    x1, y1, z1 = pos[edge[1]]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])
    edge_z.extend([z0, z1, None])

edge_trace = go.Scatter3d(x=edge_x, y=edge_y, z=edge_z, mode='lines',
                          line=dict(color='#888', width=1), hoverinfo='none')

node_x, node_y, node_z = zip(*[pos[node] for node in G.nodes()])
node_text = [f"{team}<br>Net P+R+A Change: {team_pra_changes[team]:.2f}" 
             for team in G.nodes()]

node_trace = go.Scatter3d(x=node_x, y=node_y, z=node_z, mode='markers+text',
                          marker=dict(size=10, color='#1f77b4', line_width=2),
                          text=list(G.nodes()), textposition="top center",
                          hoverinfo='text', hovertext=node_text,
                          hoverlabel=dict(bgcolor='white', font_size=12))

fig = go.Figure(data=[edge_trace, node_trace])
fig.update_layout(title='NBA Trade Deadline 2025 - Team P+R+A Changes',
                  showlegend=False, hovermode='closest',
                  scene=dict(xaxis_visible=False, yaxis_visible=False, 
                             zaxis_visible=False))

fig.show()

In [8]:
from trade_network_viz import create_trade_network

# Create the visualization
fig, team_changes = create_trade_network("trades.csv", "playerStats.csv")

# Display the plot
fig.show()

# If you want to see the PRA changes as a sorted pandas Series
import pandas as pd
changes_series = pd.Series(team_changes).sort_values(ascending=False)
print("\nTeam PRA Changes:")
print(changes_series)


Team PRA Changes:
Spurs        99.7
Jazz         75.5
Pistons      51.8
Knicks       39.4
Clippers     35.3
Hornets      31.2
Cavaliers    30.4
Pelicans     16.2
Nets         14.5
Grizzlies    11.6
Lakers       10.9
Raptors       6.9
Celtics       3.0
Rockets      -3.0
76ers        -4.0
Kings        -7.1
Pacers       -7.7
Thunder     -10.2
Warriors    -17.8
Suns        -23.0
Bulls       -35.3
Hawks       -50.7
Wizards     -53.0
Heat        -57.0
Bucks       -64.4
Mavericks   -93.2
dtype: float64
